In [ ]:
if(!require(lme4)) install.packages("lme4")
if(!require(lmerTest)) install.packages("lmerTest")
if(!require(emmeans)) install.packages("emmeans")
if(!require(pbkrtest)) install.packages("pbkrtest")
if(!require(multcompView)) install.packages("multcompView")
if(!require(multcomp)) install.packages("multcomp")
library(lme4)
library(lmerTest)
library(emmeans)
library(pbkrtest)
library(multcompView)
library(multcomp)

In [22]:
if(!require(remotes)) install.packages("remotes")
library(remotes)
if (!requireNamespace("agrivoltaics", quietly = TRUE)) {
  remotes::install_github("agronomy4future/agrivoltaics", force= TRUE)
}
library(agrivoltaics)

In [23]:
if(!require(readr)) install.packages("readr")
library(readr)
github="https://raw.githubusercontent.com/agronomy4future/raw_data_practice/refs/heads/main/agrivoltaics.csv"
df= data.frame(read_csv(url(github), show_col_types=FALSE))
df$Plot= as.factor(df$Plot)
df$Yield= as.numeric (df$Yield)
set.seed(100)
print(df[sample(nrow(df),5),])

Warning message:
“NAs introduced by coercion”


         Season Location AV_Site Genotype Plot Block  Row  Yield
202 2015 season     East      AV      cv1  101     I East 195.38
112 2016 season  MidWest Control      cv1  115    IV East 625.07
206 2015 season     East      AV      cv2  102    II East 135.86
4   2015 season  MidWest Control      cv1  109     I West 384.36
311 2016 season     East      AV      cv1  107   III East 125.11


# 1) Single crop (only 1 cultivar) with a single row

In [24]:
model= agrivoltaics(
  output= Yield,
  treatment= AV_Site,
  genotype= NULL,
  plot= NULL,
  block= Block,
  row= NULL,
  season= NULL,
  location= NULL,
  data= df
)


 MODEL FORMULA USED:
Yield ~ AV_Site + (1 | AV_Site:Block) 


 VARIANCE COMPONENTS:
 Groups        Name        Variance
 AV_Site:Block (Intercept)  113.57 
 Residual                  8086.79 

 VARIANCE COMPONENT BREAKDOWN (%):
Random [AV_Site:Block]: 1.38%
Residual: 98.62%

 TYPE III ANOVA:
Type III Analysis of Variance Table with Satterthwaite's method
         Sum Sq Mean Sq NumDF DenDF F value    Pr(>F)    
AV_Site 5311986 5311986     1 5.972  656.87 2.455e-07 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


In [25]:
post_hoc= cld (emmeans(model, ~ AV_Site), adjust= "sidak", Letters=letters, reverse= TRUE)
print(post_hoc)

 AV_Site emmean   SE   df lower.CL upper.CL .group
 Control    475 8.90 6.02      448      501  a    
 AV         152 8.88 5.98      126      179   b   

Degrees-of-freedom method: kenward-roger 
Confidence level used: 0.95 
Conf-level adjustment: sidak method for 2 estimates 
significance level used: alpha = 0.05 
NOTE: If two or more means share the same grouping symbol,
      then we cannot show them to be different.
      But we also did not show them to be the same. 


In [26]:
pairwise= contrast(emmeans(model, ~ AV_Site), method= "pairwise", adjust= "sidak")
print(summary(pairwise))

 contrast     estimate   SE df t.ratio p.value
 AV - Control     -322 12.6  6 -25.629  <.0001

Degrees-of-freedom method: kenward-roger 


# 2) Single crop (only 1 cultivar) with multiple rows

In [27]:
model= agrivoltaics(
  output= Yield,
  treatment= AV_Site,
  genotype= NULL,
  plot= NULL,
  block= Block,
  row= Row,
  season= NULL,
  location= NULL,
  data= df
)


 MODEL FORMULA USED:
Yield ~ AV_Site + Row + AV_Site:Row + (1 | AV_Site:Block) + (1 |      Block:Row) 


 VARIANCE COMPONENTS:
 Groups        Name        Variance
 Block:Row     (Intercept)  228.72 
 AV_Site:Block (Intercept)  103.48 
 Residual                  6525.07 

 VARIANCE COMPONENT BREAKDOWN (%):
Random [Block:Row]: 3.34%
Random [AV_Site:Block]: 1.51%
Residual: 95.16%

 TYPE III ANOVA:
Type III Analysis of Variance Table with Satterthwaite's method
             Sum Sq Mean Sq NumDF   DenDF  F value    Pr(>F)    
AV_Site     4187365 4187365     1   4.851 641.7345 2.422e-06 ***
Row           87937   43969     2   7.812   6.7384   0.01992 *  
AV_Site:Row  304664  152332     2 300.813  23.3456 3.758e-10 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


In [28]:
post_hoc= cld (emmeans(model, ~ AV_Site:Row), adjust= "sidak", Letters=letters, reverse= TRUE)
print(post_hoc)

 AV_Site Row    emmean   SE   df lower.CL upper.CL .group
 Control East      524 13.7 14.7      483      566  a    
 Control West      471 13.6 14.4      429      512  a    
 Control Middle    385 16.9 34.0      338      433   b   
 AV      Middle    184 16.9 34.0      136      231    c  
 AV      East      157 13.6 14.4      116      199    c  
 AV      West      131 13.6 14.4       90      173    c  

Degrees-of-freedom method: kenward-roger 
Confidence level used: 0.95 
Conf-level adjustment: sidak method for 6 estimates 
P value adjustment: sidak method for 15 tests 
significance level used: alpha = 0.05 
NOTE: If two or more means share the same grouping symbol,
      then we cannot show them to be different.
      But we also did not show them to be the same. 


In [29]:
pairwise= contrast(emmeans(model, ~ AV_Site:Row), method= "pairwise", adjust= "sidak")
print(summary(pairwise))

 contrast                      estimate   SE   df t.ratio p.value
 AV East - Control East          -366.6 16.0 14.3 -22.851  <.0001
 AV East - AV Middle              -26.2 20.5 21.7  -1.280  0.9731
 AV East - Control Middle        -227.8 21.7 23.3 -10.485  <.0001
 AV East - AV West                 26.0 17.8 12.6   1.459  0.9376
 AV East - Control West          -313.2 19.2 14.4 -16.283  <.0001
 Control East - AV Middle         340.3 21.8 23.5  15.638  <.0001
 Control East - Control Middle    138.8 20.5 21.9   6.756  <.0001
 Control East - AV West           392.6 19.3 14.6  20.364  <.0001
 Control East - Control West       53.3 17.9 12.7   2.983  0.1502
 AV Middle - Control Middle      -201.6 21.4 42.5  -9.402  <.0001
 AV Middle - AV West               52.3 20.5 21.7   2.550  0.2428
 AV Middle - Control West        -287.0 21.7 23.3 -13.210  <.0001
 Control Middle - AV West         253.8 21.7 23.3  11.684  <.0001
 Control Middle - Control West    -85.4 20.5 21.7  -4.167  0.0061
 AV West -

# 3) Single crop (only 1 cultivar) with a single row in different seasons

In [30]:
model= agrivoltaics(
  output= Yield,
  treatment= AV_Site,
  genotype= NULL,
  plot= NULL,
  block= Block,
  row= NULL,
  season= Season,
  location= NULL,
  data= df
)


 MODEL FORMULA USED:
Yield ~ AV_Site + Season + AV_Site:Season + (1 | AV_Site:Block) +      (1 | Block:Season) 


 VARIANCE COMPONENTS:
 Groups        Name        Variance
 AV_Site:Block (Intercept)   79.198
 Block:Season  (Intercept)  186.689
 Residual                  6384.063

 VARIANCE COMPONENT BREAKDOWN (%):
Random [AV_Site:Block]: 1.19%
Random [Block:Season]: 2.81%
Residual: 96%

 TYPE III ANOVA:
Type III Analysis of Variance Table with Satterthwaite's method
                Sum Sq Mean Sq NumDF   DenDF F value    Pr(>F)    
AV_Site        5531194 5531194     1   4.048 866.407 7.103e-06 ***
Season          180918  180918     1   4.990  28.339  0.003149 ** 
AV_Site:Season  126660  126660     1 305.787  19.840 1.182e-05 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


In [31]:
post_hoc= cld (emmeans(model, ~ AV_Site:Season), adjust= "sidak", Letters=letters, reverse= TRUE)
print(post_hoc)

 AV_Site Season      emmean   SE   df lower.CL upper.CL .group
 Control 2016 season    529 12.1 11.4      494      565  a    
 Control 2015 season    419 12.1 11.6      384      455   b   
 AV      2016 season    167 12.1 11.4      132      203    c  
 AV      2015 season    137 12.1 11.4      102      173    c  

Degrees-of-freedom method: kenward-roger 
Confidence level used: 0.95 
Conf-level adjustment: sidak method for 4 estimates 
P value adjustment: sidak method for 6 tests 
significance level used: alpha = 0.05 
NOTE: If two or more means share the same grouping symbol,
      then we cannot show them to be different.
      But we also did not show them to be the same. 


In [32]:
pairwise= contrast(emmeans(model, ~ AV_Site:Season), method= "pairwise", adjust= "sidak")
print(summary(pairwise))

 contrast                                  estimate   SE    df t.ratio p.value
 AV 2015 season - Control 2015 season        -282.1 14.2  9.92 -19.937  <.0001
 AV 2015 season - AV 2016 season              -30.2 15.9  9.55  -1.902  0.4237
 AV 2015 season - Control 2016 season        -392.1 17.1 11.44 -22.923  <.0001
 Control 2015 season - AV 2016 season         251.9 17.1 11.51  14.700  <.0001
 Control 2015 season - Control 2016 season   -110.0 15.9  9.62  -6.899  0.0003
 AV 2016 season - Control 2016 season        -361.8 14.1  9.82 -25.637  <.0001

Degrees-of-freedom method: kenward-roger 
P value adjustment: sidak method for 6 tests 


# 4) Single crop (only 1 cultivar) with multiple rows in different seasons

In [13]:
model= agrivoltaics(
  output= Yield,
  treatment= AV_Site,
  genotype= NULL,
  plot= NULL,
  block= Block,
  row= Row,
  season= Season,
  location= NULL,
  data= df
)


 MODEL FORMULA USED:
Yield ~ AV_Site + Season + Row + AV_Site:Row + AV_Site:Season +      AV_Site:Season:Row + (1 | AV_Site:Block) + (1 | Block:Season) +      (1 | Block:Row) 


 VARIANCE COMPONENTS:
 Groups        Name        Variance
 Block:Row     (Intercept)  288.37 
 Block:Season  (Intercept)  201.88 
 AV_Site:Block (Intercept)  116.12 
 Residual                  4794.84 

 VARIANCE COMPONENT BREAKDOWN (%):
Random [Block:Row]: 5.34%
Random [Block:Season]: 3.74%
Random [AV_Site:Block]: 2.15%
Residual: 88.77%

 TYPE III ANOVA:
Type III Analysis of Variance Table with Satterthwaite's method
                    Sum Sq Mean Sq NumDF   DenDF  F value    Pr(>F)    
AV_Site            2705378 2705378     1   7.986 564.2265 1.075e-08 ***
Season              135148  135148     1   4.803  28.1862 0.0035706 ** 
Row                  64544   32272     2   7.385   6.7306 0.0216731 *  
AV_Site:Row         117245   58623     2 290.935  12.2262 7.973e-06 ***
AV_Site:Season       60494   60494     

In [14]:
post_hoc= cld (emmeans(model, ~ AV_Site:Row), adjust= "sidak", Letters=letters, reverse= TRUE)
print(post_hoc)

NOTE: Results may be misleading due to involvement in interactions



 AV_Site Row    emmean   SE   df lower.CL upper.CL .group
 Control East      523 14.2 14.0    479.4      567  a    
 Control West      471 14.2 13.8    427.2      514  a    
 Control Middle    385 16.6 25.8    337.9      433   b   
 AV      Middle    184 16.6 25.8    136.4      231    c  
 AV      East      157 14.2 13.8    114.0      201    c  
 AV      West      131 14.2 13.8     87.9      175    c  

Results are averaged over the levels of: Season 
Degrees-of-freedom method: kenward-roger 
Confidence level used: 0.95 
Conf-level adjustment: sidak method for 6 estimates 
P value adjustment: sidak method for 15 tests 
significance level used: alpha = 0.05 
NOTE: If two or more means share the same grouping symbol,
      then we cannot show them to be different.
      But we also did not show them to be the same. 


# 5) Multiple cultivars with a single row

In [15]:
model= agrivoltaics(
  output= Yield,
  treatment= AV_Site,
  genotype= Genotype,
  plot= NULL,
  block= Block,
  row= NULL,
  season= NULL,
  location= NULL,
  data= df
)


 MODEL FORMULA USED:
Yield ~ AV_Site + Genotype + AV_Site:Genotype + (1 | AV_Site:Block) 


 VARIANCE COMPONENTS:
 Groups        Name        Variance
 AV_Site:Block (Intercept)  117.62 
 Residual                  7765.12 

 VARIANCE COMPONENT BREAKDOWN (%):
Random [AV_Site:Block]: 1.49%
Residual: 98.51%

 TYPE III ANOVA:
Type III Analysis of Variance Table with Satterthwaite's method
                  Sum Sq Mean Sq NumDF   DenDF  F value    Pr(>F)    
AV_Site          5162055 5162055     1   5.974 664.7747 2.361e-07 ***
Genotype          113976  113976     1 308.998  14.6779 0.0001545 ***
AV_Site:Genotype    2295    2295     1 308.998   0.2955 0.5870798    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


In [16]:
post_hoc= cld (emmeans(model, ~ AV_Site), adjust= "sidak", Letters=letters, reverse= TRUE)
print(post_hoc)

NOTE: Results may be misleading due to involvement in interactions



 AV_Site emmean   SE   df lower.CL upper.CL .group
 Control    475 8.85 6.02      448      501  a    
 AV         152 8.83 5.98      126      178   b   

Results are averaged over the levels of: Genotype 
Degrees-of-freedom method: kenward-roger 
Confidence level used: 0.95 
Conf-level adjustment: sidak method for 2 estimates 
significance level used: alpha = 0.05 
NOTE: If two or more means share the same grouping symbol,
      then we cannot show them to be different.
      But we also did not show them to be the same. 


# 6) Multiple cultivars with multiple rows

In [17]:
model= agrivoltaics(
  output= Yield,
  treatment= AV_Site,
  genotype= Genotype,
  plot= NULL,
  block= Block,
  row= Row,
  season= NULL,
  location= NULL,
  data= df
)


 MODEL FORMULA USED:
Yield ~ AV_Site + Genotype + Row + AV_Site:Row + AV_Site:Genotype +      AV_Site:Genotype:Row + (1 | AV_Site:Block) + (1 | Block:Row) 


 VARIANCE COMPONENTS:
 Groups        Name        Variance
 Block:Row     (Intercept)  244.45 
 AV_Site:Block (Intercept)  111.65 
 Residual                  6097.75 

 VARIANCE COMPONENT BREAKDOWN (%):
Random [Block:Row]: 3.79%
Random [AV_Site:Block]: 1.73%
Residual: 94.48%

 TYPE III ANOVA:
Type III Analysis of Variance Table with Satterthwaite's method
                      Sum Sq Mean Sq NumDF   DenDF  F value    Pr(>F)    
AV_Site              2851397 2851397     1   9.128 467.6150 3.742e-09 ***
Genotype              132953  132953     1 294.729  21.8036 4.594e-06 ***
Row                    82284   41142     2   7.756   6.7470   0.02007 *  
AV_Site:Row           236106  118053     2 294.756  19.3601 1.260e-08 ***
AV_Site:Genotype        2197    2197     1 294.788   0.3603   0.54882    
AV_Site:Genotype:Row   50912   12728    

In [18]:
post_hoc= cld (emmeans(model, ~ AV_Site:Row), adjust= "sidak", Letters=letters, reverse= TRUE)
print(post_hoc)

NOTE: Results may be misleading due to involvement in interactions



 AV_Site Row    emmean   SE   df lower.CL upper.CL .group
 Control East      524 13.6 14.4      482      565  a    
 Control West      471 13.6 14.2      429      512  a    
 Control Middle    385 16.7 32.0      338      432   b   
 AV      Middle    184 16.7 32.0      137      231    c  
 AV      East      157 13.6 14.2      116      199    c  
 AV      West      131 13.6 14.2       90      173    c  

Results are averaged over the levels of: Genotype 
Degrees-of-freedom method: kenward-roger 
Confidence level used: 0.95 
Conf-level adjustment: sidak method for 6 estimates 
P value adjustment: sidak method for 15 tests 
significance level used: alpha = 0.05 
NOTE: If two or more means share the same grouping symbol,
      then we cannot show them to be different.
      But we also did not show them to be the same. 


# 7) Multiple cultivars with a single row in different seasons

In [19]:
model= agrivoltaics(
  output= Yield,
  treatment= AV_Site,
  genotype= Genotype,
  plot= Plot,
  block= Block,
  row= NULL,
  season= Season,
  location= NULL,
  data= df
)


 MODEL FORMULA USED:
Yield ~ AV_Site + Genotype + Season + AV_Site:Genotype + AV_Site:Season +      Genotype:Season + (1 | AV_Site:Block) + (1 | Block:Season) +      (1 | Block:Plot) 



boundary (singular) fit: see help('isSingular')

Warning message in agrivoltaics(output = Yield, treatment = AV_Site, genotype = Genotype, :
“️ Model fit is singular. Some variance components may be near zero or redundant.”



 VARIANCE COMPONENTS:
 Groups        Name        Variance
 Block:Plot    (Intercept)  532.74 
 Block:Season  (Intercept)  183.48 
 AV_Site:Block (Intercept)    0.00 
 Residual                  5720.90 

 VARIANCE COMPONENT BREAKDOWN (%):
Random [Block:Plot]: 8.28%
Random [Block:Season]: 2.85%
Random [AV_Site:Block]: 0%
Residual: 88.87%

 TYPE III ANOVA:
Type III Analysis of Variance Table with Satterthwaite's method
                  Sum Sq Mean Sq NumDF   DenDF  F value    Pr(>F)    
AV_Site          2892167 2892167     1  10.980 505.5441 1.563e-10 ***
Genotype           40636   40636     1  10.980   7.1030  0.022015 *  
Season            172480  172480     1   4.538  30.1492  0.003684 ** 
AV_Site:Genotype     911     911     1  10.980   0.1592  0.697538    
AV_Site:Season    127330  127330     1 296.488  22.2569 3.679e-06 ***
Genotype:Season     5898    5898     1 296.488   1.0309  0.310773    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


In [20]:
post_hoc= cld (emmeans(model, ~ AV_Site:Genotype:Season), adjust= "sidak", Letters=letters, reverse= TRUE)
print(post_hoc)

 AV_Site Genotype Season      emmean   SE   df lower.CL upper.CL .group
 Control cv2      2016 season    555 17.4 18.6    501.9      609  a    
 Control cv1      2016 season    503 17.4 18.6    449.5      557  ab   
 Control cv2      2015 season    437 17.4 18.6    383.2      490   bc  
 Control cv1      2015 season    401 17.5 19.0    347.8      455    c  
 AV      cv2      2016 season    188 17.4 18.6    134.4      242     d 
 AV      cv2      2015 season    149 17.4 18.6     95.5      203     d 
 AV      cv1      2016 season    147 17.4 18.6     93.3      200     d 
 AV      cv1      2015 season    125 17.4 18.6     71.7      179     d 

Degrees-of-freedom method: kenward-roger 
Confidence level used: 0.95 
Conf-level adjustment: sidak method for 8 estimates 
P value adjustment: sidak method for 28 tests 
significance level used: alpha = 0.05 
NOTE: If two or more means share the same grouping symbol,
      then we cannot show them to be different.
      But we also did not show the

# 8) Multiple cultivars with multiple rows in different seasons

In [38]:
model= agrivoltaics(
  output= Yield,
  treatment= AV_Site,
  genotype= Genotype,
  plot= Plot,
  block= Block,
  row= Row,
  season= Season,
  location= NULL,
  data= df
)


 MODEL FORMULA USED:
Yield ~ AV_Site + Genotype + Season + Row + AV_Site:Row + AV_Site:Genotype +      AV_Site:Season + Genotype:Season + AV_Site:Genotype:Row +      AV_Site:Season:Row + Genotype:Season:Row + (1 | AV_Site:Block) +      (1 | Block:Season) + (1 | Block:Plot) + (1 | Plot:Row) 



boundary (singular) fit: see help('isSingular')

Warning message in agrivoltaics(output = Yield, treatment = AV_Site, genotype = Genotype, :
“️ Model fit is singular. Some variance components may be near zero or redundant.”



 VARIANCE COMPONENTS:
 Groups        Name        Variance
 Plot:Row      (Intercept) 2206.547
 Block:Plot    (Intercept)   20.564
 Block:Season  (Intercept)  249.014
 AV_Site:Block (Intercept)    0.000
 Residual                  3065.545

 VARIANCE COMPONENT BREAKDOWN (%):
Random [Plot:Row]: 39.82%
Random [Block:Plot]: 0.37%
Random [Block:Season]: 4.49%
Random [AV_Site:Block]: 0%
Residual: 55.32%

 TYPE III ANOVA:
Type III Analysis of Variance Table with Satterthwaite's method
                     Sum Sq Mean Sq NumDF   DenDF  F value    Pr(>F)    
AV_Site              683286  683286     1  25.264 222.8922 4.832e-14 ***
Genotype              25054   25054     1  11.011   8.1728  0.015536 *  
Season                87093   87093     1   4.589  28.4104  0.004019 ** 
Row                   29239   14620     2  22.991   4.7690  0.018513 *  
AV_Site:Row           37444   18722     2  27.491   6.1072  0.006389 ** 
AV_Site:Genotype        218     218     1  30.072   0.0712  0.791421    
AV_Sit

In [41]:
post_hoc= cld (emmeans(model, ~ AV_Site:Row), adjust= "sidak", Letters=letters, reverse= TRUE)
print(post_hoc)

NOTE: Results may be misleading due to involvement in interactions



 AV_Site Row    emmean   SE   df lower.CL upper.CL .group
 Control East      522 18.9 24.2    468.3      577  a    
 Control West      471 18.9 24.1    416.5      525  a    
 Control Middle    385 20.1 30.9    328.7      442   b   
 AV      Middle    184 20.1 30.9    127.1      240    c  
 AV      East      157 18.9 24.1    103.3      212    c  
 AV      West      131 18.9 24.1     77.3      186    c  

Results are averaged over the levels of: Genotype, Season 
Degrees-of-freedom method: kenward-roger 
Confidence level used: 0.95 
Conf-level adjustment: sidak method for 6 estimates 
P value adjustment: sidak method for 15 tests 
significance level used: alpha = 0.05 
NOTE: If two or more means share the same grouping symbol,
      then we cannot show them to be different.
      But we also did not show them to be the same. 
